In [1]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
team = pd.read_csv("knox.csv")
opponents = pd.read_csv("knox_opponents.csv")

In [3]:
team['Opponent'] = team.Opponent.apply(lambda i : i.replace("vs\xa0", "").replace("at\xa0", ""))

advanced = team.merge(opponents, on="Opponent", suffixes=("", " OPP"))

In [4]:
advanced['Margin'] = advanced.Score.apply(lambda i : int(i.split("-")[0]) - int(i.split("-")[1]))

advanced['FGM'] = advanced['FGM/A'].apply(lambda i : float(i.split("-")[0]))
advanced['FGA'] = advanced['FGM/A'].apply(lambda i : float(i.split("-")[1]))
advanced['3PM'] = advanced['3FG/A'].apply(lambda i : float(i.split("-")[0]))
advanced['3PA'] = advanced['3FG/A'].apply(lambda i : float(i.split("-")[1]))
advanced['FTA'] = advanced['FTM/A'].apply(lambda i : float(i.split("-")[1]))
advanced['OPP FGM'] = advanced['FGM/A OPP'].apply(lambda i : float(i.split("-")[0]))
advanced['OPP FGA'] = advanced['FGM/A OPP'].apply(lambda i : float(i.split("-")[1]))
advanced['OPP 3PM'] = advanced['3FG/A OPP'].apply(lambda i : float(i.split("-")[0]))
advanced['OPP 3PA'] = advanced['3FG/A OPP'].apply(lambda i : float(i.split("-")[1]))
advanced['OPP FTA'] = advanced['FTM/A OPP'].apply(lambda i : float(i.split("-")[1]))
advanced['2PM'] = advanced['FGM'] - advanced['3PM']
advanced['2PA'] = advanced['FGA'] - advanced['3PA']
advanced['OPP 2PM'] = advanced['OPP FGM'] - advanced['OPP 3PM']
advanced['OPP 2PA'] = advanced['OPP FGA'] - advanced['OPP 3PA']

advanced.loc[len(advanced)] = advanced.sum(numeric_only=True, axis=0)

advanced['POSS'] = round(0.96 * (advanced.FGA + 
                                 advanced.TO + 
                                 0.44 * advanced.FTA) - 
                                 advanced.OFF, 1)
advanced['PPP'] = round(advanced.PTS / advanced.POSS, 2)
advanced['REB'] = advanced.TOT.astype(int)
advanced['DRB%'] = 100 * round(advanced.DEF / (advanced.DEF + advanced['OFF OPP']), 3)
advanced['ORB%'] = 100 * round(advanced.OFF / (advanced.OFF + advanced['DEF OPP']), 3)
advanced['RB%'] = 100 * round(advanced.TOT / (advanced.TOT + advanced['TOT OPP']), 3)
advanced['TO%'] = 100 * round(advanced.TO / advanced.POSS, 3)

advanced['OPP POSS'] = round(0.96 * (advanced['OPP FGA'] + 
                                     advanced['TO OPP'] + 
                                     0.44 * advanced['OPP FTA'] - 
                                     advanced['OFF OPP']), 1)
advanced['OPP PPP'] = round(advanced['PTS OPP'] / advanced['OPP POSS'], 2)
advanced['OPP REB'] = advanced['TOT OPP'].astype(int)

advanced.loc[len(advanced)-1, 'Opponent'] = "Total"
advanced.loc[len(advanced)-1, 'W/L'] = ""
advanced.loc[len(advanced)-1, 'POSS'] /= len(advanced)-1
advanced.loc[len(advanced)-1, 'OPP POSS'] /= len(advanced)-1

advanced['POSS'] = advanced['POSS'].round(1)
advanced['OPP POSS'] = advanced['OPP POSS'].round(1)
advanced['Margin'] = advanced['Margin'].astype(int)

advanced = advanced.set_index('Opponent')

In [5]:
advanced.columns

Index(['Date', 'Score', 'W/L', 'FGM/A', 'PCT', '3FG/A', 'PCT.1', 'FTM/A',
       'PCT.2', 'OFF', 'DEF', 'TOT', 'AVG', 'PF', 'AST', 'TO', 'BLK', 'STL',
       'PTS', 'AVG.1', 'Date OPP', 'Score OPP', 'W/L OPP', 'FGM/A OPP',
       'PCT OPP', '3FG/A OPP', 'PCT.1 OPP', 'FTM/A OPP', 'PCT.2 OPP',
       'OFF OPP', 'DEF OPP', 'TOT OPP', 'AVG OPP', 'PF OPP', 'AST OPP',
       'TO OPP', 'BLK OPP', 'STL OPP', 'PTS OPP', 'AVG.1 OPP', 'Margin', 'FGM',
       'FGA', '3PM', '3PA', 'FTA', 'OPP FGM', 'OPP FGA', 'OPP 3PM', 'OPP 3PA',
       'OPP FTA', '2PM', '2PA', 'OPP 2PM', 'OPP 2PA', 'POSS', 'PPP', 'REB',
       'DRB%', 'ORB%', 'RB%', 'TO%', 'OPP POSS', 'OPP PPP', 'OPP REB'],
      dtype='object')

In [6]:
advanced[['W/L', 'Margin',
          'POSS', 'OPP POSS', 
          'PPP', 'OPP PPP', 
          'REB', 'OPP REB', 'ORB%', 'DRB%', 'RB%', 
          'TO%']].to_csv("scout.csv")

In [7]:
advanced[['POSS']]

,POSS
Opponent,
Dominican (IL),73.9
Blackburn,73.8
Coe,61.4
Central (IA),82.6
Grinnell,71.8
Principia,70.8
Lawrence,72.9
Ripon,67.6
Westminster (MO),84.3


In [8]:
advanced[['W/L', 'Margin', '2PA', '3PA']].sort_values(by='3PA')

,W/L,Margin,2PA,3PA
Opponent,,,,
Eureka,W,8,65.0,6.0
Ripon,W,13,41.0,9.0
Westminster (MO),L,-2,68.0,12.0
Coe,L,-2,38.0,14.0
Lawrence,W,7,44.0,14.0
Grinnell,W,9,55.0,15.0
Principia,L,-11,46.0,18.0
Central (IA),W,16,53.0,25.0
Blackburn,W,45,67.0,26.0


In [9]:
advanced[['W/L', 'Margin', '3FG/A', 'PCT.1']].sort_values(by=['PCT.1'])


,W/L,Margin,3FG/A,PCT.1
Opponent,,,,
Coe,L,-2,3-14,0.214
Principia,L,-11,4-18,0.222
Blackburn,W,45,6-26,0.231
Central (IA),W,16,6-25,0.240
Grinnell,W,9,4-15,0.267
Dominican (IL),W,31,8-28,0.286
Lawrence,W,7,4-14,0.286
Westminster (MO),L,-2,4-12,0.333
Ripon,W,13,4-9,0.444


In [10]:
advanced[['W/L', 'Margin', '3FG/A OPP', 'PCT.1 OPP']].sort_values(by='PCT.1 OPP')

,W/L,Margin,3FG/A OPP,PCT.1 OPP
Opponent,,,,
Dominican (IL),W,31,4-24,0.167
Westminster (MO),L,-2,3-15,0.200
Blackburn,W,45,5-23,0.217
Lawrence,W,7,6-24,0.250
Ripon,W,13,5-19,0.263
Grinnell,W,9,6-22,0.273
Principia,L,-11,9-27,0.333
Coe,L,-2,7-20,0.350
Central (IA),W,16,9-25,0.360


In [11]:
advanced[['W/L', 'Margin', 'ORB%', 'TO', 'TO OPP', 'PCT.1']].sort_values(by='PCT.1')


,W/L,Margin,ORB%,TO,TO OPP,PCT.1
Opponent,,,,,,
Coe,L,-2,36.0,13.0,9.0,0.214
Principia,L,-11,28.9,14.0,16.0,0.222
Blackburn,W,45,46.7,6.0,25.0,0.231
Central (IA),W,16,41.3,19.0,23.0,0.240
Grinnell,W,9,35.6,14.0,16.0,0.267
Dominican (IL),W,31,36.0,13.0,23.0,0.286
Lawrence,W,7,42.9,23.0,21.0,0.286
Westminster (MO),L,-2,51.2,25.0,24.0,0.333
Ripon,W,13,28.0,22.0,18.0,0.444


In [12]:
advanced[['W/L', 'Margin', 'ORB%', 'TO', 'PCT.1', 'TO OPP', '3FG/A OPP']].sort_values(by='PCT.1')


,W/L,Margin,ORB%,TO,PCT.1,TO OPP,3FG/A OPP
Opponent,,,,,,,
Coe,L,-2,36.0,13.0,0.214,9.0,7-20
Principia,L,-11,28.9,14.0,0.222,16.0,9-27
Blackburn,W,45,46.7,6.0,0.231,25.0,5-23
Central (IA),W,16,41.3,19.0,0.240,23.0,9-25
Grinnell,W,9,35.6,14.0,0.267,16.0,6-22
Dominican (IL),W,31,36.0,13.0,0.286,23.0,4-24
Lawrence,W,7,42.9,23.0,0.286,21.0,6-24
Westminster (MO),L,-2,51.2,25.0,0.333,24.0,3-15
Ripon,W,13,28.0,22.0,0.444,18.0,5-19


In [13]:
advanced

,Date,Score,W/L,FGM/A,PCT,3FG/A,PCT.1,FTM/A,PCT.2,OFF,DEF,TOT,AVG,PF,AST,TO,BLK,STL,PTS,AVG.1,Date OPP,Score OPP,W/L OPP,FGM/A OPP,PCT OPP,3FG/A OPP,PCT.1 OPP,FTM/A OPP,PCT.2 OPP,OFF OPP,DEF OPP,TOT OPP,AVG OPP,PF OPP,AST OPP,TO OPP,BLK OPP,STL OPP,PTS OPP,AVG.1 OPP,Margin,FGM,FGA,3PM,3PA,FTA,OPP FGM,OPP FGA,OPP 3PM,OPP 3PA,OPP FTA,2PM,2PA,OPP 2PM,OPP 2PA,POSS,PPP,REB,DRB%,ORB%,RB%,TO%,OPP POSS,OPP PPP,OPP REB
Opponent,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Dominican (IL),11/08/22,76-45,W,25-70,0.357,8-28,0.286,18-29,0.621,18.0,36.0,54.0,54.0,14.0,17.0,13.0,5.0,12.0,76.0,76.0,11/08/22,45-76,L,18-67,0.269,4-24,0.167,5-8,0.625,14.0,32.0,46.0,46.0,20.0,14.0,23.0,3.0,6.0,45.0,45.0,31,25.0,70.0,8.0,28.0,29.0,18.0,67.0,4.0,24.0,8.0,17.0,42.0,14.0,43.0,73.9,1.03,54,72.0,36.0,54.0,17.6,76.3,0.59,46
Blackburn,11/11/22,87-42,W,34-93,0.366,6-26,0.231,13-16,0.813,28.0,31.0,59.0,56.5,12.0,18.0,6.0,5.0,12.0,87.0,81.5,11/11/22,42-87,L,17-56,0.304,5-23,0.217,3-8,0.375,10.0,32.0,42.0,44.0,16.0,11.0,25.0,3.0,2.0,42.0,43.5,45,34.0,93.0,6.0,26.0,16.0,17.0,56.0,5.0,23.0,8.0,28.0,67.0,12.0,33.0,73.8,1.18,59,75.6,46.7,58.4,8.1,71.5,0.59,42
Coe,11/15/22,58-60,L,21-52,0.404,3-14,0.214,13-19,0.684,9.0,25.0,34.0,49.0,14.0,13.0,13.0,3.0,4.0,58.0,73.7,11/15/22,60-58,W,24-60,0.400,7-20,0.350,5-12,0.417,11.0,16.0,27.0,38.3,16.0,10.0,9.0,2.0,8.0,60.0,49.0,-2,21.0,52.0,3.0,14.0,19.0,24.0,60.0,7.0,20.0,12.0,18.0,38.0,17.0,40.0,61.4,0.94,34,69.4,36.0,55.7,21.2,60.7,0.99,27
Central (IA),11/23/22,89-73,W,34-78,0.436,6-25,0.240,15-20,0.750,19.0,34.0,53.0,50.0,17.0,22.0,19.0,5.0,8.0,89.0,77.5,11/23/22,73-89,L,26-62,0.419,9-25,0.360,12-19,0.632,7.0,27.0,34.0,37.3,18.0,15.0,23.0,2.0,11.0,73.0,55.0,16,34.0,78.0,6.0,25.0,20.0,26.0,62.0,9.0,25.0,19.0,28.0,53.0,17.0,37.0,82.6,1.08,53,82.9,41.3,60.9,23.0,82.9,0.88,34
Grinnell,11/29/22,75-66,W,29-70,0.414,4-15,0.267,13-17,0.765,16.0,28.0,44.0,48.8,16.0,19.0,14.0,3.0,11.0,75.0,77.0,11/29/22,66-75,L,24-64,0.375,6-22,0.273,12-17,0.706,13.0,29.0,42.0,38.2,18.0,15.0,16.0,3.0,7.0,66.0,57.2,9,29.0,70.0,4.0,15.0,17.0,24.0,64.0,6.0,22.0,17.0,25.0,55.0,18.0,42.0,71.8,1.04,44,68.3,35.6,51.2,19.5,71.5,0.92,42
Principia,12/02/22,63-74,L,23-64,0.359,4-18,0.222,13-21,0.619,13.0,30.0,43.0,47.8,20.0,11.0,14.0,4.0,7.0,63.0,74.7,12/02/22,74-63,W,25-67,0.373,9-27,0.333,15-23,0.652,16.0,32.0,48.0,39.8,17.0,11.0,16.0,2.0,7.0,74.0,60.0,-11,23.0,64.0,4.0,18.0,21.0,25.0,67.0,9.0,27.0,23.0,19.0,46.0,16.0,40.0,70.8,0.89,43,65.2,28.9,47.3,19.8,74.0,1.00,48
Lawrence,12/07/22,65-58,W,20-58,0.345,4-14,0.286,21-31,0.677,18.0,33.0,51.0,48.3,28.0,17.0,23.0,7.0,7.0,65.0,73.3,12/07/22,58-65,L,19-61,0.311,6-24,0.250,14-24,0.583,14.0,24.0,38.0,39.6,27.0,11.0,21.0,4.0,8.0,58.0,59.7,7,20.0,58.0,4.0,14.0,31.0,19.0,61.0,6.0,24.0,24.0,16.0,44.0,13.0,37.0,72.9,0.89,51,70.2,42.9,57.3,31.6,75.4,0.77,38
Ripon,12/10/22,67-54,W,27-50,0.540,4-9,0.444,9-13,0.692,7.0,23.0,30.0,46.0,17.0,18.0,22.0,10.0,7.0,67.0,72.5,12/10/22,54-67,L,19-55,0.345,5-19,0.263,11-19,0.579,12.0,18.0,30.0,38.4,14.0,15.0,18.0,1.0,13.0,54.0,59.0,13,27.0,50.0,4.0,9.0,13.0,19.0,55.0,5.0,19.0,19.0,23.0,41.0,14.0,36.0,67.6,0.99,30,65.7,28.0,50.0,32.5,66.6,0.81,30
Westminster (MO),12/12/22,82-84,L,34-80,0.425,4-12,0.333,10-13,0.769,22.0,28.0,50.0,46.4,18.0,21.0,25.0,9.0,16.0,82.0,73.6,12/12/22,84-82,W,29-72,0.403,3-15,0.200,23-30,0.767,13.0,21.0,34.0,37.9,14.0,10.0,24.0,2.0,14.0,84.0,61.8,-2,34.0,80.0,4.0,12.0,13.0,29.0,72.0,3.0,15.0,30.0,30.0,68.0,26.0,57.0,84.3,0.97,50,68.3,51.2,59.5,29.7,92.4,0.91,34
